In [28]:
import cv2
import os
import time
import numpy as np
from scipy import ndimage, signal
from matplotlib import pyplot as plt
import shutil

In [29]:
kSC = np.array([[-1 / 8, -1 / 8, -1 / 8],  # from class material
                [-1 / 8, 1, -1 / 8],
                [-1 / 8, -1 / 8, -1 / 8]])

def convert(img, target_type_min, target_type_max, target_type):
    imin = img.min()
    imax = img.max()

    a = (target_type_max - target_type_min) / (imax - imin)
    b = target_type_max - a * imax
    new_img = (a * img + b).astype(target_type)
    return new_img
def toGray(im, im_L=10, im_W=10, contrast=1.0):
    # normalize
    gray = np.zeros([im_L, im_W])
    for j in range(im_L):
        for i in range(im_W):
            gray[j][i] = max(im[j][i]) * contrast
    gray = convert(gray, 0, 255, np.uint8)
    return gray
def threshold(frame):
    # for j in range(len(frame)):
    #     for i in range(len(frame[0])):
    #         print(frame[j][i])
    #
    # print("---------------------------------")
    out = np.full_like(frame, 128)
    out[frame < -4] = 0
    out[frame > 4] = 255
    return out
def SpacialContrast(imGray):
    conv_gray = signal.convolve2d(imGray, kSC, 'same')
    fSCgray = np.array(threshold(conv_gray), dtype=np.uint8)
    return fSCgray

In [30]:
training_set = np.empty([120*9+1, 10, 10])
src_set = np.empty(shape=120*9+1, dtype=object)
baseDir = 'ShapeGroups'
idx = 0
for filename in os.listdir(baseDir):
    folder = baseDir + '/' + filename
    for imagepath in os.listdir(folder):
        imagepathFull = folder + '/' + imagepath
        training_set[idx] = toGray(cv2.imread(imagepathFull))
        src_set[idx] = imagepathFull
        # print(training_set[idx])
        idx += 1

C:\Users\ereij\AppData\Local\Temp\ipykernel_12372\2113857434.py:9: RuntimeWarning: divide by zero encountered in scalar divide
  a = (target_type_max - target_type_min) / (imax - imin)
C:\Users\ereij\AppData\Local\Temp\ipykernel_12372\2113857434.py:11: RuntimeWarning: invalid value encountered in add
  new_img = (a * img + b).astype(target_type)
C:\Users\ereij\AppData\Local\Temp\ipykernel_12372\2113857434.py:11: RuntimeWarning: invalid value encountered in cast
  new_img = (a * img + b).astype(target_type)


In [ ]:
for sample, dir in zip(training_set, src_set):
    plt.imshow(sample)
    plt.show()
    print(dir)
    time.sleep(.01)

In [ ]:
cv2.imshow("", cv2.resize(training_set[0], [50, 50], interpolation=cv2.INTER_LINEAR))

cv2.waitKey(0)

In [ ]:
# testing some maths
sh4_22 = training_set[1]
sh0_01 = training_set[2]
sh0_22 = training_set[3]

print(sum(sum(abs(sh4_22.astype(int) - sh0_22.astype(int)))))
print(sum(sum(abs(sh4_22.astype(int) - sh0_01.astype(int)))))

In [ ]:
fSC4_22 = SpacialContrast(sh4_22)
fSC0_01 = SpacialContrast(sh0_01)
fSC0_22 = SpacialContrast(sh0_22)

print(sum(sum(abs(fSC4_22.astype(int)/255 - fSC0_01.astype(int)/255))))
print(sum(sum(abs(fSC4_22.astype(int)/255 - fSC0_22.astype(int)/255))))
print(sum(sum(abs(fSC0_22.astype(int)/255 - fSC0_01.astype(int)/255))))

fig = plt.figure()
fig.add_subplot(2, 3, 1)
plt.imshow(fSC4_22)

fig.add_subplot(2, 3, 2)
plt.imshow(fSC0_01)

fig.add_subplot(2, 3, 3)
plt.imshow(fSC0_22)

fig.add_subplot(2, 3, 4)
plt.imshow(abs(fSC4_22.astype(int)/255 - fSC0_01.astype(int)/255))

fig.add_subplot(2, 3, 5)
plt.imshow(abs(fSC4_22.astype(int)/255 - fSC0_22.astype(int)/255))

fig.add_subplot(2, 3, 6)
plt.imshow(abs(fSC0_22.astype(int)/255 - fSC0_01.astype(int)/255))
plt.show()

In [ ]:
fSC0_20 = toGray(cv2.imread("ShapeGroups/shape5/0_20.png"))
fSC4_02 = toGray(cv2.imread("ShapeGroups/shape15/4_12.png"))

print(sum(sum(abs(fSC0_20.astype(int)/255 - fSC4_02.astype(int)/255))))
plt.imshow((abs(fSC0_20.astype(int)/255 - fSC4_02.astype(int)/255)))
plt.show()

In [ ]:
# clean dataset
# toFind = ["0_11", "103_12", "105_02", "98_00", "87_12",
#           "74_02", "65_11", "63_10", "62_02", "52_00",
#           "38_22", "2_11", "21_02", "17_22", "115_01",
#           "112_11", "111_10", "107_02"]
# toMove = "ShapeGroups/shape5"
toFind = ["5_22", "4_00"]
toMove = "ShapeGroups/shape7"
idx = 0
for sample in toFind:
    im = "shape7/" + sample
    for idx in range(len(src_set)):
        dir = src_set[idx]
        if dir.find(im) != -1:
            print(dir)
            # plt.imshow(training_set[idx])
            # plt.show()
            # shutil.move(dir, toMove)